In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from __future__ import print_function
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

from sklearn.model_selection import validation_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [2]:
train = pd.read_csv('C:\Pedro\workspace\Acc-challenge\data_v3_3.csv')
#train = pd.read_csv('C:\Pedro\workspace\Acc-challenge\data_v3_2.csv')
#test = pd.read_csv('C:\Pedro\workspace\Acc-challenge\\test_data_v3.csv')
#train = train.sample(frac=1).reset_index(drop=True)

#concated_names = train['PlayerName'].append(test['PlayerName']).unique()
#pname_mapping = dict(zip(sorted(concated_names), range(0, len(sorted(concated_names)) + 1)))
pname_mapping = dict(zip(sorted(train['PlayerName'].unique()), range(0, len(sorted(train['PlayerName'].unique())) + 1)))
aname_mapping = dict(zip(sorted(train['ActionName'].unique()), range(0, len(sorted(train['ActionName'].unique())) + 1)))
atype_mapping = dict(zip(sorted(train['ActionTypeDesc'].unique()), range(0, len(sorted(train['ActionTypeDesc'].unique())) + 1)))
spos_mapping = dict(zip(sorted(train['StartingPositionDesc'].unique()), range(0, len(sorted(train['StartingPositionDesc'].unique())) + 1)))
epos_mapping = dict(zip(sorted(train['EndPositionDesc'].unique()), range(0, len(sorted(train['EndPositionDesc'].unique())) + 1)))
stad_mapping = dict(zip(sorted(train['StadiumName'].unique()), range(0, len(sorted(train['StadiumName'].unique())) + 1)))
team_mapping = dict(zip(sorted(train['Team'].unique()), range(0, len(sorted(train['Team'].unique())) + 1)))
zone_mapping = dict(zip(sorted(train['ZoneDescription'].unique()), range(0, len(sorted(train['ZoneDescription'].unique())) + 1)))
summary_mapping = dict(zip(sorted(train['Summary'].unique()), range(0, len(sorted(train['Summary'].unique())) + 1)))

train['HomeTeam'].apply(lambda x: team_mapping.update(zip([x],[len(team_mapping.keys())])) 
                         if team_mapping.has_key(x) == False else None)

train['AwayTeam'].apply(lambda x: team_mapping.update(zip([x],[len(team_mapping.keys())])) 
                         if team_mapping.has_key(x) == False else None)

train['PlayerName_Val'] = train['PlayerName'].map(pname_mapping).astype(int)
train['ActionName_Val'] = train['ActionName'].map(aname_mapping).astype(int)
train['ActionType_Val'] = train['ActionTypeDesc'].map(atype_mapping).astype(int)
train['StartingPositionDesc_Val'] = train['StartingPositionDesc'].map(spos_mapping).astype(int)
train['EndPositionDesc_Val'] = train['EndPositionDesc'].map(epos_mapping).astype(int)
train['StadiumName_Val'] = train['StadiumName'].map(stad_mapping).astype(int)
train['Team_Val'] = train['Team'].map(team_mapping).astype(int)
train['HomeTeam_Val'] = train['HomeTeam'].map(team_mapping).astype(int)
train['AwayTeam_Val'] = train['AwayTeam'].map(team_mapping).astype(int)
train['ZoneDescription_Val'] = train['ZoneDescription'].map(zone_mapping).astype(int)
train['Summary_Val'] = train['Summary'].map(summary_mapping).astype(int)

np.random.seed(0)
train = train.reindex(np.random.permutation(train.index)) #Shuffling

X_target = train['ActionName_Val'].values

X_train = train.drop(['ActionName_Val','ActionName','PlayerName','StartingPositionDesc',
                      'EndPositionDesc','Summary','RecordID','ActionTypeDesc','StadiumName',
                        'Team','HomeTeam','AwayTeam','ZoneDescription','Visibility','Pressure',
                         'Humidity','DewPoint','CloudCover','Temperature'], axis=1).values


In [3]:
# Splitting data into train, cv and test sets using 60%, 20%, 20% proportions
#X_train, X_cv, y_train, y_cv = train_test_split(X_train, X_target, test_size=0.4, random_state=42)
#X_cv, X_test, y_cv, y_test = train_test_split(X_cv, y_cv, test_size=0.5, random_state=42)

# Splitting data into train, cv and test sets using 80%, 10%, 10% proportions
X_train, X_cv, y_train, y_cv = train_test_split(X_train, X_target, test_size=0.2, random_state=42)
X_cv, X_test, y_cv, y_test = train_test_split(X_cv, y_cv, test_size=0.5, random_state=42)

# Using only Train and CV: 80-20
#X_train, X_cv, y_train, y_cv = train_test_split(X_train, X_target, test_size=0.2, random_state=42)

In [5]:
scaler = StandardScaler()
scaler.fit(X_train)
X_normalized = scaler.transform(X_train)
X_cv_normalized = scaler.transform(X_cv)
X_test_normalized = scaler.transform(X_test)

# Set the parameters by cross-validation

rate_param = np.logspace(-3,1,num=4,base=2.0)
print(rate_param)

#tuned_parameters = [{'max_features': [4,6,8,10,12,14,16,20,22,24,'auto'], 
#                     'n_estimators': [10, 50, 100, 200, 400]}] #RandomForest

tuned_parameters = [{'min_samples_leaf': range(1, 70, 5), 
                     'min_samples_split': range(2, 20, 2)}] #RandomForest

#tuned_parameters = [{'learning_rate': [1.4,1.6,1.2,1.0,0.8,0.6,0.4],
#                     'n_estimators': [10, 25, 50, 75, 100, 200, 400, 600]}] #AdaBoost

#tuned_parameters = [{'learning_rate': [0.31498026],'n_estimators': [200]}] #AdaBoost

#tuned_parameters = [{'learning_rate': rate_param,'n_estimators': [25,50]}] #AdaBoost

scores = ['precision', 'recall']

[ 0.125       0.31498026  0.79370053  2.        ]


In [47]:
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    ## Used to fit the Stump estimator parameters
    #base_estimator=RandomForestClassifier(max_depth=None, min_samples_split=2, 
    #                                      min_samples_leaf=1, n_jobs=4, class_weight='balanced')
    
    base_estimator=RandomForestClassifier(max_depth=None, max_features=14, n_estimators=10, n_jobs=4, class_weight='balanced')
    
    ## Used to fit AdaBoost
    #base_estimator=RandomForestClassifier(max_depth=None, min_samples_split=2, min_samples_leaf=1, n_jobs=4, 
    #                       class_weight='balanced', max_features=14, n_estimators=10)
    
    ada_real = AdaBoostClassifier(base_estimator=base_estimator, algorithm="SAMME.R")

    clf = GridSearchCV(base_estimator, tuned_parameters, cv=5, scoring='%s_macro' % score)
    #clf = GridSearchCV(ada_real, tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf.fit(X_normalized, y_train)
    #clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    #y_true, y_pred = y_test, clf.predict(X_test)
    y_true, y_pred = y_cv, clf.predict(X_cv_normalized)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'min_samples_split': 8, 'min_samples_leaf': 51}

Grid scores on development set:

0.549 (+/-0.109) for {'min_samples_split': 2, 'min_samples_leaf': 1}
0.572 (+/-0.159) for {'min_samples_split': 4, 'min_samples_leaf': 1}
0.587 (+/-0.160) for {'min_samples_split': 6, 'min_samples_leaf': 1}
0.578 (+/-0.065) for {'min_samples_split': 8, 'min_samples_leaf': 1}
0.583 (+/-0.069) for {'min_samples_split': 10, 'min_samples_leaf': 1}
0.574 (+/-0.149) for {'min_samples_split': 12, 'min_samples_leaf': 1}
0.579 (+/-0.095) for {'min_samples_split': 14, 'min_samples_leaf': 1}
0.571 (+/-0.081) for {'min_samples_split': 16, 'min_samples_leaf': 1}
0.585 (+/-0.079) for {'min_samples_split': 18, 'min_samples_leaf': 1}
0.590 (+/-0.059) for {'min_samples_split': 2, 'min_samples_leaf': 6}
0.573 (+/-0.122) for {'min_samples_split': 4, 'min_samples_leaf': 6}
0.587 (+/-0.114) for {'min_samples_split': 6, 'min_samples_leaf':

In [6]:
ada_tmp = AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=None, min_samples_split=3, min_samples_leaf=2, 
                                n_jobs=4, class_weight='balanced', max_features=8, n_estimators=10), 
                             algorithm="SAMME.R", n_estimators=500, learning_rate=0.0675)

#ada_tmp = AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=None, min_samples_split=8, min_samples_leaf=51, 
#                                n_jobs=4, class_weight='balanced', max_features=14, n_estimators=10), 
#                             algorithm="SAMME.R", n_estimators=500, learning_rate=0.0675)

ada_tmp.fit(X_normalized,y_train)
#clf.param_grid = {'n_estimators': 200, 'learning_rate': 0.1}
#print(clf.score(X_test_normalized,y_test))

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features=8,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=2, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          learning_rate=0.0675, n_estimators=500, random_state=None)

In [7]:
ada_tmp.score(X_test_normalized, y_test)

0.96250000000000002

In [51]:
ada_tmp.feature_importances_

array([ 0.04380797,  0.05622355,  0.03498273,  0.04082949,  0.12295559,
        0.10337428,  0.02993285,  0.03057933,  0.02438626,  0.02455615,
        0.03117653,  0.03084531,  0.09665527,  0.07359508,  0.06643316,
        0.00848542,  0.01526771,  0.00669005,  0.03094512,  0.047321  ,
        0.0241514 ,  0.02955126,  0.0041441 ,  0.0231104 ])

In [45]:
ex = RandomForestClassifier(max_depth=None, min_samples_split=2, min_samples_leaf=1, n_jobs=4, 
                            class_weight='balanced', max_features=14, n_estimators=10)

print(ex.fit(X_normalized, y_train))

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features=14,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [46]:
print(ex.score(X_test_normalized,y_test))

0.7625


In [12]:
np.random.seed(0)
X, y = X_train, X_target
indices = np.arange(y.shape[0])
np.random.shuffle(indices)
X, y = X[indices], y[indices]

In [13]:
param_range = [1,2,4,8,12,13]
train_scores, test_scores = validation_curve(RandomForestClassifier(n_estimators=1000), X_train, X_target, param_name="max_features", param_range=param_range,
                                             cv=5, scoring="accuracy", n_jobs=1)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

print("Train scores \n\n %s \n" % train_scores)
print("Valid scores \n\n %s" % test_scores)

plt.title("Validation Curve with SVM")
plt.xlabel("$\gamma$")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

Train scores 

 [[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]] 

Valid scores 

 [[ 0.73584906  0.74213836  0.71698113  0.6918239   0.76582278]
 [ 0.72955975  0.72955975  0.72955975  0.71069182  0.75316456]
 [ 0.75471698  0.73584906  0.73584906  0.71698113  0.74683544]
 [ 0.7672956   0.73584906  0.72955975  0.71698113  0.75949367]
 [ 0.74842767  0.71698113  0.73584906  0.71069182  0.75949367]
 [ 0.74213836  0.73584906  0.73584906  0.70440252  0.76582278]]


In [82]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


title = "Learning Curves Random Forest Classifier 400 / 13"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

estimator = RandomForestClassifier(n_estimators=400, max_features=13, max_depth=None, min_samples_split=1)
plot_learning_curve(estimator, title, X_train, X_target, ylim=(0.5, 1.01), cv=cv, n_jobs=4)

title = "Learning Curves AdaBoost"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = AdaBoostClassifier(
    base_estimator=RandomForestClassifier(max_depth=None, min_samples_split=1, n_estimators=100, max_features=12),
    learning_rate=0.8,
    n_estimators=600,
    algorithm="SAMME.R")
plot_learning_curve(estimator, title, X_train, X_target, (0.5, 1.01), cv=cv, n_jobs=4)

plt.show()

In [8]:
test = pd.read_csv('C:\Pedro\workspace\Acc-challenge\\test_data_v3_3.csv')
#test = pd.read_csv('C:\Pedro\workspace\Acc-challenge\\test_data_v3_2.csv')
#test = test.sample(frac=1).reset_index(drop=True)

test['PlayerName'].apply(lambda x: pname_mapping.update(zip([x],[len(pname_mapping.keys())])) 
                         if pname_mapping.has_key(x) == False else None)

test['StartingPositionDesc'].apply(lambda x: spos_mapping.update(zip([x],[len(spos_mapping.keys())])) 
                         if spos_mapping.has_key(x) == False else None)

test['EndPositionDesc'].apply(lambda x: epos_mapping.update(zip([x],[len(epos_mapping.keys())])) 
                         if epos_mapping.has_key(x) == False else None)

test['ActionTypeDesc'].apply(lambda x: atype_mapping.update(zip([x],[len(atype_mapping.keys())])) 
                         if atype_mapping.has_key(x) == False else None)

test['Summary'].apply(lambda x: summary_mapping.update(zip([x],[len(summary_mapping.keys())])) 
                         if summary_mapping.has_key(x) == False else None)

test['StadiumName'].apply(lambda x: stad_mapping.update(zip([x],[len(stad_mapping.keys())])) 
                         if stad_mapping.has_key(x) == False else None)

test['Team'].apply(lambda x: team_mapping.update(zip([x],[len(team_mapping.keys())])) 
                         if team_mapping.has_key(x) == False else None)

test['ZoneDescription'].apply(lambda x: zone_mapping.update(zip([x],[len(zone_mapping.keys())])) 
                         if zone_mapping.has_key(x) == False else None)

test['StadiumName_Val'] = test['StadiumName'].map(stad_mapping).astype(int)
test['Team_Val'] = test['Team'].map(team_mapping).astype(int)
test['HomeTeam_Val'] = test['HomeTeam'].map(team_mapping).astype(int)
test['AwayTeam_Val'] = test['AwayTeam'].map(team_mapping).astype(int)
test['ZoneDescription_Val'] = test['ZoneDescription'].map(zone_mapping).astype(int)

test['PlayerName_Val'] = test['PlayerName'].map(pname_mapping).astype(int)
test['StartingPositionDesc_Val'] = test['StartingPositionDesc'].map(spos_mapping).astype(int)
test['ActionTypeDesc_Val'] = test['ActionTypeDesc'].map(atype_mapping).astype(int)
test['EndPositionDesc_Val'] = test['EndPositionDesc'].map(epos_mapping).astype(int)
test['Summary_Val'] = test['Summary'].map(summary_mapping).astype(int)


Test_ids = test['RecordID']
Test_predict = test.drop(['RecordID','PlayerName','StartingPositionDesc',
                              'EndPositionDesc','Summary','ActionTypeDesc','StadiumName',
                                'Team','HomeTeam','AwayTeam','ZoneDescription','Visibility','Pressure',
                                 'Humidity','DewPoint','CloudCover','Temperature'], axis=1).values

#Test_predict = test.drop(['PlayerName','StartingPositionDesc','EndPositionDesc','Summary',
#                             'Summary_Val','EndPositionDesc_Val','End_Y','ActionTypeDesc'], axis=1)

# No need to shuffle prediction set (we are not fitting it)
#np.random.seed(0)
#Test_sorted = Test_predict.reindex(np.random.permutation(Test_predict.index))
#X_test = Test_sorted.drop(['RecordID'], axis=1).values

# Apply same transformation to test data (normalization)
test_set_normalized = scaler.transform(Test_predict)

In [64]:
ada_real = AdaBoostClassifier(
    base_estimator=RandomForestClassifier(max_depth=None, min_samples_split=1, n_estimators=100, max_features=12),
    learning_rate=0.8,
    n_estimators=400,
    algorithm="SAMME.R")
    
ada_real.fit(X_normalized, X_target)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=12, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=1, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          learning_rate=0.8, n_estimators=400, random_state=None)

In [55]:
#pred_y = ada_real.predict(X_cv_normalized)
pred_y = ada_tmp.predict(X_cv_normalized)
print(pred_y)

print('\n')

target_names = ['Goal Kick Missed', 'Goal Kick Success']
print(classification_report(y_cv, pred_y, target_names=target_names))

[1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1
 0 1 0 1 1 0 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 0 0 1
 1 0 0 1 1]


                   precision    recall  f1-score   support

 Goal Kick Missed       0.52      0.63      0.57        19
Goal Kick Success       0.88      0.82      0.84        60

      avg / total       0.79      0.77      0.78        79



In [9]:
#pred_y_test = ada_real.predict(test_set_normalized)
pred_y_test = ada_tmp.predict(test_set_normalized) # Already fitted earlier

print(pred_y_test)

[1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1]


In [10]:
reverse_map = {0: 'Goal Kick Missed', 1: 'Goal Kick Success'}

result = pd.DataFrame({'RowId': Test_ids})
result['Prediction'] = pred_y_test.T
result['Prediction'] = result['Prediction'].map(reverse_map)
result

result.to_excel('PedroCastanha_AdaBoost_Recall_Normalized_NewMap_RandomForest_NewParams_Score.xlsx', index=False)

In [18]:
forest = ExtraTreesClassifier(n_estimators=250, random_state=0)
forest.fit(X_train, X_target)

importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices], color="r", yerr=std[indices], align="center")
plt.xticks(range(X_train.shape[1]), indices)
plt.xlim([-1, X_train.shape[1]])
plt.show()

Feature ranking:
1. feature 1 (0.127705)
2. feature 0 (0.121489)
3. feature 13 (0.079950)
4. feature 11 (0.078692)
5. feature 7 (0.077209)
6. feature 9 (0.074318)
7. feature 5 (0.069823)
8. feature 8 (0.065933)
9. feature 6 (0.064566)
10. feature 2 (0.063508)
11. feature 3 (0.050709)
12. feature 4 (0.050206)
13. feature 10 (0.048022)
14. feature 12 (0.027871)
